<a href="https://colab.research.google.com/github/beanb36/Acoustic-Modelling-project/blob/main/Assignment4InClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold,GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import GradientBoostingClassifier

import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, roc_curve, auc


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
HouseDF=pd.read_csv('gdrive/My Drive/housing.csv')


Mounted at /content/gdrive


In [ ]:
# Extract features and target
X = HouseDF[['housing_median_age', 'total_rooms',
'total_bedrooms', 'population', 'households', 'median_income','ocean_proximity']]
X = pd.get_dummies(X, columns=['ocean_proximity'], drop_first=True)
y = HouseDF['median_house_value']

In [ ]:
# Handle missing values by filling NaNs with the mean of the column
X = X.fillna(X.mean())


In [ ]:
# Standardize the features after filling missing values
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)




In [ ]:
percentile_25 = y_train.quantile(0.25)
percentile_75 = y_train.quantile(0.75)

y_train_cat = pd.cut(y_train, bins=[0, percentile_25, percentile_75, float('inf')],
                      labels=['poor', 'med', 'rich'])
y_test_cat = pd.cut(y_test, bins=[0, percentile_25, percentile_75, float('inf')],
                     labels=['poor', 'med', 'rich'])




In [ ]:
# Train a classification model (example using DecisionTreeClassifier)
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train_cat)

# Get predictions from the classification model
dt_pred_cat = dt_classifier.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Train the best Random Forest model
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train_cat)

rf_pred_cat = rf_classifier.predict(X_test)

In [ ]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Define base models
base_models = [
  ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
  ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42)),
  ('svc', SVC(probability=True, random_state=42))
]
# Define meta-learner
meta_model = LogisticRegression()
# Build Stacking Classifier
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)


# Train the model
stacking_clf.fit(X_train, y_train_cat)  # Changed to y_train_cat for classification

# Predictions
stacking_pred_cat = stacking_clf.predict(X_test)

TypeError: StackingClassifier.__init__() got an unexpected keyword argument 'estimators'

# Neuer Abschnitt

In [ ]:
cm = confusion_matrix(y_test_cat, stacking_pred_cat, labels=['poor', 'med', 'rich'])

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['poor', 'med', 'rich'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix for Stacking Classifier')
plt.show()

In [ ]:
cm = confusion_matrix(y_test_cat, dt_pred_cat, labels=['poor', 'med', 'rich'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['poor', 'med', 'rich'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.show()

In [ ]:
cm = confusion_matrix(y_test_cat, rf_pred_cat, labels=['poor', 'med', 'rich'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['poor', 'med', 'rich'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix for Random Forest')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Decision Tree
dt_accuracy = accuracy_score(y_test_cat, dt_pred_cat)
dt_precision = precision_score(y_test_cat, dt_pred_cat, average='weighted')
dt_recall = recall_score(y_test_cat, dt_pred_cat, average='weighted')
dt_f1 = f1_score(y_test_cat, dt_pred_cat, average='weighted')

# Random Forest
rf_accuracy = accuracy_score(y_test_cat, rf_pred_cat)
rf_precision = precision_score(y_test_cat, rf_pred_cat, average='weighted')
rf_recall = recall_score(y_test_cat, rf_pred_cat, average='weighted')
rf_f1 = f1_score(y_test_cat, rf_pred_cat, average='weighted')

# Stacking Classifier
stacking_accuracy = accuracy_score(y_test_cat, stacking_pred_cat)
stacking_precision = precision_score(y_test_cat, stacking_pred_cat, average='weighted')
stacking_recall = recall_score(y_test_cat, stacking_pred_cat, average='weighted')
stacking_f1 = f1_score(y_test_cat, stacking_pred_cat, average='weighted')

print("Decision Tree:")
print(f"  Accuracy: {dt_accuracy:.4f}")
print(f"  Precision: {dt_precision:.4f}")
print(f"  Recall: {dt_recall:.4f}")
print(f"  F1-score: {dt_f1:.4f}")

print("\nRandom Forest:")
print(f"  Accuracy: {rf_accuracy:.4f}")
print(f"  Precision: {rf_precision:.4f}")
print(f"  Recall: {rf_recall:.4f}")
print(f"  F1-score: {rf_f1:.4f}")

print("\nStacking Classifier:")
print(f"  Accuracy: {stacking_accuracy:.4f}")
print(f"  Precision: {stacking_precision:.4f}")
print(f"  Recall: {stacking_recall:.4f}")
print(f"  F1-score: {stacking_f1:.4f}")

